In [ ]:
#import directory and netcdf libs
import datetime as dt
from datetime import datetime
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import pandas
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
#import plotly
#import plotly.plotly as py
#from plotly.graph_objs import *
#!pip install geopy  #to install geopy use !pip install geopy but only run once
from geopy.distance import geodesic 
#set plotly credentials
#plotly.tools.set_credentials_file(username='cgentemann', api_key='7uYsbupVsAjAScD0Hb0O')

####################you will need to change some paths here!#####################
#list of input files
filename_aviso='f:/data/project_data/NASA_biophysical/aviso/eddy_trajectory_19930101_20170106.nc'   #From AVISO  website
filename_cpr='f:/data/project_data/NASA_biophysical/cpr_data/All CPR Sample catalogue.xlsx'
filename_distance_land='f:/data/sat_data/distance_to_land_25km.nc' 
#list of output files
filename_northpac_eddies='f:/data/project_data/NASA_biophysical/aviso/eddy_trajectory_19930101_20170106_north_pacific_2020_10_06.nc'
filename_cpr_eddy='f:/data/project_data/NASA_biophysical/collocated_data/cpr/eddy_cpr_data_north_pacific_2020_10_06.nc'
filename_eddy='f:/data/project_data/NASA_biophysical/collocated_data/cpr/eddy_ranking_data_north_pacific_2020_10_06.nc'
#################################################################################



In [ ]:
#open the netcdf file full eddy database from AVISO
nc_fid = Dataset(filename_aviso, 'r')  
# Extract data from NetCDF file
time = nc_fid.variables['time'][:]
lats = nc_fid.variables['latitude'][:]  
lons = nc_fid.variables['longitude'][:]
speed_average = nc_fid.variables['speed_average'][:]
amplitude = nc_fid.variables['amplitude'][:]
speed_radius = nc_fid.variables['speed_radius'][:]
cyclonic_type = nc_fid.variables['cyclonic_type'][:]
track = nc_fid.variables['track'][:]   #eddy identification number  0 - 272045
observation_number = nc_fid.variables['observation_number'][:]
nc_fid.close()
max_eddy_index=len(track)  
print('total number of eddies in database:',max_eddy_index)


In [ ]:
#use datetime delta feature to find year and day
#make sure lon is 0-360 for collocations since we are doing north pacific & cross dateline
date_1950 = dt.datetime(1950,1,1,0,0,0) # start date is 1/1/1950
year=[0] * (max_eddy_index)
idayjl=[0] * (max_eddy_index)
for t in range(0,max_eddy_index):
    tem_date=date_1950+dt.timedelta(days=float(time[t]))  #create new time array that can be queried for year etc
    year[t]=tem_date.year
    idayjl[t]=tem_date.timetuple().tm_yday
    if lons[t] < -180.:
        lons[t]=lons[t]+360.
    if t < 5:
        print(year[t],idayjl[t])   
print('max and min lons', max(lons), min(lons))

In [ ]:
# this code identifies the eddies that have any data in 2000 to 2017 in the north pacific region
# the code sorts by eddy identification number after this so that if an eddy just passes through the north pacific
# we still save all the eddy location and time information

#find the maximum eddy identification integer
max_eddy_identification=max(track)
#create an array with 0/1 flag 0= default; 1 = it was in region of interest
npac_eddy=[0] * (max_eddy_identification+1)
for index in range(0,max_eddy_index):
    if year[index]>=2000 and lats[index]>=33 and (lons[index]>=140 and lons[index]<=245):
        index_npac=track[index]
        npac_eddy[index_npac]=1    #the array npac_eddy now has 1 if that eddy was in north pacific'
print('total number of eddies in north pacific',sum(npac_eddy),len(npac_eddy))



In [ ]:
#create a new dataset where you only have eddies from ones that are in the north pacific after 2000
print(npac_eddy[0],npac_eddy[272045-1])
time_new=[]
lats_new=[]
lons_new=[]
amplitude_new=[]
speed_radius_new=[]
speed_average_new=[]
cyclonic_type_new=[]
track_new=[]
observation_number_new=[]
year_new=[]
idayjl_new=[]
for index in range(0,max_eddy_index):
    index_npac=track[index]  #check the track id for the eddy & if it is in north pacific
    if index_npac>272045:
        print(index_npac)
    if npac_eddy[index_npac]==1:
        time_new.append(time[index])
        lats_new.append(lats[index])
        lons_new.append(lons[index])
        amplitude_new.append(amplitude[index])
        speed_average_new.append(speed_average[index])
        speed_radius_new.append(speed_radius[index])
        cyclonic_type_new.append(cyclonic_type[index])
        track_new.append(track[index])
        observation_number_new.append(observation_number[index])
        year_new.append(year[index])
        idayjl_new.append(idayjl[index])
        

In [ ]:
print(len(time_new))

In [ ]:
#eddy_id is from AVISO so it starts at 79110 and goes to 272035 but not
#sequentially since I downsampled the data.  Here I calucalte a new eddy
#index that goes from 1 to N sequentially
#this isn't used here, but could be used to go through all the eddies

#first go through data and save a new eddy id for each first observation
ilen_eddy=len(lons_new)
track_npac=0
print(max(track_new))
track_npac_new=[0] * (max(track_new)+1)
track_npac_new_array=[0] * (ilen_eddy+1)
for index in range(0,ilen_eddy):
    if observation_number_new[index]==0:
        track_npac=track_npac+1
        track_npac_new[track_new[index]]=track_npac
max_new_eddy_number = track_npac
new_length_eddy = [0] * (max_new_eddy_number+1)
#now go through data and assign new track id for all observations
for index in range(0,ilen_eddy):
    itrack=track_npac_new[track_new[index]]
    track_npac_new_array[index]=itrack
#find all the eddies that match new eddy number to determine length of each eddy
for ieddy_num in range(1,max_new_eddy_number):
    new_list=[index for index, x in enumerate(track_npac_new_array) if x==ieddy_num] 
    new_length_eddy[ieddy_num]=len(new_list)

In [ ]:
track_npac_new_array_length=[0]*(ilen_eddy)
for index in range(0,ilen_eddy):
    ieddy=track_npac_new_array[index]
    ilen=new_length_eddy[ieddy]
    track_npac_new_array_length[index]=ilen
print('done')

In [ ]:
da = xr.DataArray(lons_new, dims=('z'))
ds = xr.Dataset({'Longitude':lons_new})
ds['Latitude']=xr.DataArray(lats_new, dims=('z'))
ds['time']=xr.DataArray(time_new, dims=('z'))
ds['amplitude']=xr.DataArray(amplitude_new, dims=('z'))
ds['speed_average']=xr.DataArray(speed_average_new, dims=('z'))
ds['speed_radius']=xr.DataArray(speed_radius_new, dims=('z'))
ds['cyclonic_type']=xr.DataArray(, dims=('z'))
ds['track']=xr.DataArray(, dims=('z'))
ds['track_total_days']=xr.DataArray(, dims=('z'))
ds['observation_number']=xr.DataArray(, dims=('z'))
ds['year']=xr.DataArray(, dims=('z'))
ds['idyjl']=xr.DataArray(, dims=('z'))


In [ ]:
inum_obs=len(idayjl_new)

f = Dataset(filename_northpac_eddies,'w', format='NETCDF4') 
#tempgrp = f.createGroup('Eddy_data')
f.createDimension('z', inum_obs)
lons_netcdf = f.createVariable('Longitude', 'f4', 'z')
lats_netcdf = f.createVariable('Latitude', 'f4', 'z')  
time_netcdf = f.createVariable('time', 'i4', 'z')
amplitude_netcdf = f.createVariable('amplitude', 'f4', 'z')
speed_average_netcdf = f.createVariable('speed_average', 'f4', 'z')
speed_radius_netcdf = f.createVariable('speed_radius', 'f4', 'z')
cyclonic_type_netcdf = f.createVariable('cyclonic_type', 'i4', 'z')
track_netcdf = f.createVariable('track', 'i4', 'z')
track_days_netcdf = f.createVariable('track_total_days', 'i4', 'z')
observation_number_netcdf = f.createVariable('observation_number', 'i2', 'z')
year_netcdf = f.createVariable('year', 'f4', 'z')
idayjl_netcdf = f.createVariable('idayjl', 'f4', 'z')
lons_netcdf[:] = lons_new
lats_netcdf[:] = lats_new
time_netcdf[:] = time_new
amplitude_netcdf[:] = amplitude_new
cyclonic_type_netcdf[:] = cyclonic_type_new
speed_average_netcdf[:] = speed_average_new
speed_radius_netcdf[:] = speed_radius_new
track_netcdf[:] = track_new
observation_number_netcdf[:] = observation_number_new
year_netcdf[:] =year_new
idayjl_netcdf[:] = idayjl_new
track_days_netcdf[:]=track_npac_new_array_length
lons_netcdf.units = 'degrees east'
lats_netcdf.units = 'degrees north'
time_netcdf.units = 'days since 1950-01-01 00:00:00 UTC'
amplitude_netcdf.units = 'cm'
speed_average_netcdf.units = 'cm/s'
speed_radius_netcdf.units = 'km'
cyclonic_type_netcdf.units = 'cyclonic -1; anti-cyclonic +1'
track_netcdf.units = 'eddy identification number'
track_days_netcdf.units = 'total number of days eddy identified'
observation_number_netcdf.units = 'observation sequence number, days from start'
f.close()

In [ ]:
ds = xr.open_dataset(filename_northpac_eddies)
ds.close()
ds

In [ ]:
#read in CPR data excell file using pandas library
df = pandas.read_excel(filename_cpr)
#print the column names
print(df.columns)
#get the values for a given column
cpr_sample_id = df['Sample ID'].values
cpr_sample_day = df['day'].values
cpr_sample_month = df['month'].values
cpr_sample_year = df['year'].values
cpr_sample_lat = df['lat'].values
cpr_sample_lon = df['Long'].values
cpr_sample_proc = df['Already processed?'].values

In [ ]:
ilen_cpr=len(cpr_sample_proc)
cpr_sample_jday=[0] * (ilen_cpr)
for index in range(0,ilen_cpr):
    tem_time = dt.datetime(cpr_sample_year[index],cpr_sample_month[index],cpr_sample_day[index])
    tt = tem_time.timetuple()
    cpr_sample_jday[index]=tt.tm_yday
    #print(cpr_sample_jday[index],cpr_sample_month[index],cpr_sample_day[index])

In [ ]:
ilen_cpr=len(cpr_sample_proc)
print(len(cpr_sample_proc))
#change lon -180 180 to 0 360
for index in range(0,ilen_cpr):
    if cpr_sample_lon[index]<0.:
        cpr_sample_lon[index]=cpr_sample_lon[index]+360.
ilen_eddy=len(lons_new)
for index in range(0,ilen_eddy):
    if lons_new[index]<.0:
        lons_new[index]=lons_new[index]+360.
if cpr_sample_proc[0]=='Yes':
    print(cpr_sample_proc[0:5])
print(min(cpr_sample_lon),max(cpr_sample_lon))

In [ ]:
#read in distance from land file and calculate all the cpr data distance from land  
nc_fid = Dataset(filename_distance_land, 'r')  
# Extract data from NetCDF file
distance_to_land_km = nc_fid.variables['dist_land'][:]
nc_fid.close()
print(min(cpr_sample_lon),max(cpr_sample_lon))
print(type(distance_to_land_km))
print(distance_to_land_km.shape)
print(distance_to_land_km[160][200])  #checked == to dist_land(201,161) in matlab
#lon goes from -180 to 180
cpr_dist_from_land=[0]*(ilen_cpr)
print(len(cpr_sample_lon),ilen_cpr)
for index in range(0,ilen_cpr):
    lon_tem=cpr_sample_lon[index]
    if lon_tem>180:
        lon_tem=lon_tem-360.
    ilon=int(round((lon_tem-179.875)/.25))
    ilat=int(round((cpr_sample_lat[index]+89.875)/.25))
    if ilon>1440:
        print(ilon,ilat,type(ilon),type(ilat))
        print(cpr_sample_lon[index])
    if ilon<0:
        ilon=ilon+1440
    #print(ilon,ilat,distance_to_land_km[ilat][ilon])
    cpr_dist_from_land[index]=distance_to_land_km[ilat][ilon]
#print(cpr_dist_from_land[0:10])

In [ ]:
print(cpr_sample_lat[0:3])
print(lats_new[0:3])

In [ ]:
import geopy
## print(len(cpr_sample_lat),ilen_cpr)
#print(len(lats_new),ilen_eddy)
save_eddy_index=[0] * (ilen_cpr)
save_eddy_distance=[0] * (ilen_cpr)
save_eddy_radius=[0] * (ilen_cpr)
print(ilen_cpr,ilen_eddy)
for index_cpr in range(0,ilen_cpr):
    arclen_to_eddy=100000  #set to a high value
    for index_eddy in range(0,ilen_eddy):
        if cpr_sample_year[index_cpr]==year_new[index_eddy] and cpr_sample_jday[index_cpr]==idayjl_new[index_eddy]:
            coords_1 = (cpr_sample_lat[index_cpr], cpr_sample_lon[index_cpr])  #cpr sample location
            coords_2 = (lats_new[index_eddy], lons_new[index_eddy])  #eddy location
            arclen_temp = geopy.distance.vincenty(coords_1, coords_2).km
            if arclen_temp<arclen_to_eddy:
                arclen_to_eddy=arclen_temp
                save_eddy_index[index_cpr]=index_eddy
                save_eddy_distance[index_cpr]=arclen_temp
                save_eddy_radius[index_cpr]=speed_radius_new[index_eddy]  
#                save_observation_number[index_cpr]=observation_number_new[index_eddy]
    index_eddy=save_eddy_index[index_cpr]
    #print(cpr_sample_day[index_cpr],idayjl_new[index_eddy],index_eddy)
print(save_eddy_index[0:5])
print(save_eddy_distance[0:5])
print(save_eddy_radius[0:5])

In [ ]:
save_total_number_eddy_days=[0] * (ilen_cpr)
save_total_number_cruises=[0] * (ilen_cpr)
for index_cpr in range(0,ilen_cpr):
    index_eddy=save_eddy_index[index_cpr] #get eddy index
    track_id=track_new[index_eddy]          #get eddy track id
    #search full N.Pac eddy archive for days
    indices = [i for i, x in enumerate(track_new) if x==track_id]
    save_total_number_eddy_days[index_cpr]=len(indices)
    #for i in range(0,len(indices)):
    #    print(year_new[indices[i]])
    #now indices has all the times the cpr data crossed this eddy
    tem_data=[0]*len(indices)
    for ind in range(0,len(indices)):
        ind2=indices[ind]
        tem_data[ind]=year_new[ind2]
    for lyr in range(2000,2018):
        #year_array=year_new[indices]
        indices = [i for i, x in enumerate(tem_data) if x==lyr]
        if len(indices)>0:
            save_total_number_cruises[index_cpr]= \
            save_total_number_cruises[index_cpr]+1

In [ ]:
#create cpr list of eddy track id
save_eddy_track=[0]*ilen_cpr
for index_cpr in range(0,ilen_cpr):
    index_eddy=save_eddy_index[index_cpr] #get eddy index
    track_id=track_new[index_eddy]          #get eddy track id
    save_eddy_track[index_cpr]=track_id
#now file unique track id
print(max(save_eddy_track))
cpr_unique_eddy=[0]
cpr_unique_eddy_index_cpr=[0]
cpr_unique_eddy_flag_array=[0]*(max(save_eddy_track)+1)
for index_cpr in range(0,ilen_cpr):
    track_id=save_eddy_track[index_cpr]   #get eddy index
    if cpr_unique_eddy_flag_array[track_id]==0:
        cpr_unique_eddy.append(track_id)
        cpr_unique_eddy_index_cpr.append(index_cpr)
        cpr_unique_eddy_flag_array[track_id]=1
    else:
        cpr_unique_eddy_flag_array[track_id]=cpr_unique_eddy_flag_array[track_id]+1
ilen=len(cpr_unique_eddy)
cpr_unique_eddy=cpr_unique_eddy[1:ilen+1]
cpr_unique_eddy_index_cpr=cpr_unique_eddy_index_cpr[1:ilen+1]
print(cpr_unique_eddy[0:10])
#now in cpr_unique_eddy is a list of all the unique eddy tha tthe cpr data colloates with
#now get all sorts of info about the eddy to help distingish what we want to look at
ilen=len(cpr_unique_eddy)
cpr_unique_eddy_collocations=[0]*ilen
for index in range(0,ilen):
    track_id=cpr_unique_eddy[index]
    index_cpr=cpr_unique_eddy_index_cpr[index]
    cpr_unique_eddy_collocations[index]=cpr_unique_eddy_flag_array[track_id]
#output a file to go along with cpr unique eddy info
filename_eddy='F:\data\eddy\eddy_randing_data_north_pacific.nc'
f = Dataset(filename_eddy,'w', format='NETCDF4') #'w' stands for write
#tempgrp = f.createGroup('eddy_data')
f.createDimension('z', ilen)
lons_netcdf = f.createVariable('cpr_unique_eddy_collocations', 'i4', 'z')
lats_netcdf = f.createVariable('cpr_unique_eddy', 'i4', 'z')  
time_netcdf = f.createVariable('cpr_unique_eddy_index_cpr', 'i4', 'z')
lons_netcdf[:] = cpr_unique_eddy_collocations
lats_netcdf[:] = cpr_unique_eddy
time_netcdf[:] = cpr_unique_eddy_index_cpr
lons_netcdf.units = 'eddy number of cpr collocations'
lats_netcdf.units = 'unique eddy track id'
time_netcdf.units = 'cpr index for eddy id'
f.close()

In [ ]:
#output a file to go along with cpr data in netcdf
#file contains the collocated eddy index, distance, and radius, distance from land
f = Dataset(filename_cpr_eddy,'w', format='NETCDF4') #'w' stands for write
#tempgrp = f.createGroup('CPR_data')
f.createDimension('z', ilen_cpr)
lons_netcdf = f.createVariable('save_eddy_index', 'i4', 'z')
lats_netcdf = f.createVariable('save_eddy_distance', 'f4', 'z')  
time_netcdf = f.createVariable('save_eddy_radius', 'f4', 'z')
dlnd_netcdf = f.createVariable('cpr_dist_from_land', 'f4', 'z')
lons_netcdf[:] = save_eddy_index
lats_netcdf[:] = save_eddy_distance
time_netcdf[:] = save_eddy_radius
dlnd_netcdf[:] = cpr_dist_from_land
lons_netcdf.units = 'eddy index number'
lats_netcdf.units = 'distance to eddy in km'
time_netcdf.units = 'radius of eddy in km'
dlnd_netcdf.units = 'distance from land in km'
f.close()

In [ ]:
print(save_eddy_index[0:2],cpr_dist_from_land[0:2])
print(save_eddy_distance[0:2],save_eddy_radius[0:2])

In [ ]:
#plt.hist(amplitude_new)
#plt.title("")
#plt.xlabel("Value")
#plt.ylabel("Frequency")
#fig = plt.gcf()
#plot_url = py.plot_mpl(fig, filename='mpl-basic-histogram')
# the histogram of the data
#n, bins, patches = plt.hist(amplitude_new, 50, normed=1, facecolor='green', alpha=0.75)
# add a 'best fit' line
#print(type(bins))
# the histogram of the data with histtype='step'
n, bins, patches = plt.hist(amplitude_new, 50, normed=1, histtype='stepfilled')
plt.setp(patches, 'facecolor', 'g', 'alpha', 0.75)
# add a line showing the expected distribution
l = plt.plot(bins)
plt.show()

#pyplot.set_cmap('hot')

#layout = Layout(title="Eddy Amplitude Histogram",
#                xaxis=dict(title='Eddy Amplitude (cm)'),
#                yaxis=dict(title='Eddy Radius (km)'))
#fig = Figure(data=data, layout=layout)
#py.iplot(data, filename='basic_bar')
#create a table
#table = ff.create_table(df)
#py.iplot(table, filename='jupyter/table1')


#for index_cpr in range(0,ilen_cpr):
#    index_eddy=save_eddy_index[index_cpr]
    

